In [2]:
#r "nuget: Plotly.NET, 2.0.0-beta8"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta8"
#r "nuget: Deedle, 2.3.0"
#r "nuget: FSharp.Data, 4.1.0"
#r "System.Xml.Linq.dll"


In [3]:
open System
open Deedle
open FSharp.Data

open Plotly.NET
open Plotly.NET.Interactive

In [4]:
type WorldData = XmlProvider<"http://api.worldbank.org/countries/indicators/NY.GDP.PCAP.CD?date=2010:2010">
let indUrl = "http://api.worldbank.org/countries/indicators/"

let getData year indicator =
    let query =
        [("per_page","1000"); ("date",sprintf "%d:%d" year year)]
    let data = Http.RequestString(indUrl + indicator, query)
    let xml = WorldData.Parse(data)
    let orNaN value =
        defaultArg (Option.map float value) nan
    series [ for d in xml.Datas -> d.Country.Value, orNaN d.Value ]

printfn "Loaded World Bank data"


Loaded World Bank data


In [5]:
let wb = WorldBankData.GetDataContext()
let inds = wb.Countries.World.Indicators
let code = inds.``CO2 emissions (kt)``.IndicatorCode

let co2000 = getData 2000 code
let co2010 = getData 2010 code

let change = (co2010 - co2000) / co2000 * 100.0
    

In [6]:
Chart.ChoroplethMap(
  change.Keys, change.Values,
  Locationmode=StyleParam.LocationFormat.CountryNames
) |> Chart.withTitle "CO2 emissions (world)"


<!-- Plotly chart will be drawn inside this DIV -->